# Example attacks against Large Language Models with TextAttack

In this notebook, we will show you how to use Textattack, a library to test the robustness of LLMs.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zangobot/teaching_material/blob/HEAD/06-LLMSecurity.ipynb)

In [1]:
!pip install textattack tensorflow tensorflow_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 61.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 23.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.7/445.7 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import transformers

An attack in TextAttack consists of four parts:

- **Goal function**: determines if the attack is successful or not: evaluates how well a perturbed attacked_text object is achieving a specified goal. A common goal function is **targeted**/**untargeted classification**, where the attack tries to perturb an input to change its classification.

- **Transformation**: takes a text input and transforms it, for example by replacing words or phrases with similar ones, while trying not to change the meaning. Paraphrase and synonym substitution are two broad classes of transformations.

- **Constraints**: determine whether or not a given transformation is valid. Transformations don't perfectly preserve syntax or semantics, so additional constraints can increase the probability that these qualities are preserved from the source to adversarial example. There are many types of constraints: overlap constraints that measure edit distance, syntactical  constraints check part-of-speech and grammar errors, and semantic constraints like language models and sentence encoders.

- **Search method**: explores the space of possible transformations within the defined constraints and attempt to find a successful perturbation which satisfies the goal function. Some examples are greedy search with word importance ranking, beam search, brute force, and genetic algorithm.


In this way, attacking an NLP model can be framed as a combinatorial search problem. The attacker must search within all potential transformations to find a sequence of transformations that generate a successful adversarial example.

TextAttack provides a set of **Attack Recipes** that assemble attacks from the literature from these four components. We can exploit an attack recipe or build an attack from scratch.


# Example 1: Attack Recipe

In [3]:
from textattack.datasets import HuggingFaceDataset

# load the dataset from HuggingFace
dataset = HuggingFaceDataset("rotten_tomatoes", None, "test")

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:44: SyntaxWarning: invalid escape sequence '\.'
  re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._%\-]+)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:46: SyntaxWarning: invalid escape s

README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

textattack: Loading datasets dataset rotten_tomatoes, split test.


In [4]:
# print some examples
print(dataset[0])
print(dataset[1])
print(dataset[2])
print(dataset[3])
print(dataset[4])

(OrderedDict({'text': 'lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .'}), 1)
(OrderedDict({'text': 'consistently clever and suspenseful .'}), 1)
(OrderedDict({'text': 'it\'s like a " big chill " reunion of the baader-meinhof gang , only these guys are more harmless pranksters than political activists .'}), 1)
(OrderedDict({'text': 'the story gives ample opportunity for large-scale action and suspense , which director shekhar kapur supplies with tremendous skill .'}), 1)
(OrderedDict({'text': 'red dragon " never cuts corners .'}), 1)


In [5]:
from textattack.models.wrappers import HuggingFaceModelWrapper

# load a pre-trained BERT-based model for sequence classification from the HuggingFace model hub
model = transformers.AutoModelForSequenceClassification.from_pretrained(
    "textattack/distilbert-base-uncased-rotten-tomatoes"
)

# load the corresponding tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(
    "textattack/distilbert-base-uncased-rotten-tomatoes"
)

# TextAttack requires both the model and the tokenizer to be wrapped by a ModelWrapper class that implements the forward pass operation given a list of input texts
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

config.json:   0%|          | 0.00/496 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [6]:
print("\nMODEL: ", model)
print("\nTOKENIZER: ", tokenizer)
print("\nWRAPPER: ", model_wrapper)


MODEL:  DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplac

Let's use TextFooler, which operates at word-level.

The main idea behind **TextFooler** is to generate adversarial examples by iteratively replacing or modifying words in the input text to maximize the model's prediction error while minimizing the perturbation's perceptibility to human readers. The framework utilizes semantic and syntactic similarity metrics to guide the perturbation process, ensuring that the modified text remains grammatically and semantically coherent.

In [7]:
from textattack.attack_recipes import TextFoolerJin2019

# load the attack
attack = TextFoolerJin2019.build(model_wrapper)

textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.

  0%|          | 0.00/481M [00:00<?, ?B/s]
  0%|          | 2.19M/481M [00:00<00:21, 21.8MB/s]
  1%|          | 4.37M/481M [00:00<00:36, 13.0MB/s]
  1%|          | 5.87M/481M [00:00<00:39, 12.1MB/s]
  2%|▏         | 7.25M/481M [00:00<00:37, 12.6MB/s]
  2%|▏         | 8.62M/481M [00:00<00:37, 12.7MB/s]
  2%|▏         | 9.94M/481M [00:00<00:38, 12.4MB/s]
  2%|▏         | 12.0M/481M [00:00<00:31, 14.8MB/s]
  3%|▎         | 16.6M/481M [00:00<00:19, 23.9MB/s]
  4%|▍         | 19.1M/481M [00:01<00:19, 23.1MB/s]
  4%|▍         | 21.5M/481M [00:01<00:20, 22.2MB/s]
  5%|▌         | 26.1M/481M [00:01<00:15, 28.9MB/s]
  7%|▋         | 31.5M/481M [00:01<00:12, 36.1MB/s]
  8%|▊         | 37.2M/481M [00:01<00:10, 41.9MB/s]
  9%|▉         | 43.0M/481M [00:01<00:09, 46.7MB/s]
 10%|█         | 48.2M/481M [00:01<00:08, 48.5MB/s]
 11%|█         | 53.3M/481M [00:01<00:08, 49.0MB/s]
 12%|█▏        | 58.7M/481M [00:01<0

In [8]:
print("\nATTACK: ", attack)


ATTACK:  Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis

By printing the attack we can check the definition of the four components: search method, goal function, transformation, constraints.

Let's run the attack.

In [9]:
from textattack import Attacker
from textattack import AttackArgs

attack_args = AttackArgs(num_examples = 10)

attacker = Attacker(attack, dataset, attack_args)

TextAttack attacks iterate through a dataset and, for each correctly predicted sample, search for an adversarial perturbation. If an example is incorrectly predicted to begin with, it is not attacked (Skipped), since the input already fools the model.

In [10]:
attack_results = attacker.attack_dataset()

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

  0%|          | 0/10 [00:00<?, ?it/s]

LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger_eng[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger_eng/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


To visualize the results, we are logging AttackResult objects using a CSVLogger. This logger stores all attack results in a dataframe, which we can easily access and display. Since we set color_method to 'html', the attack results will display their differences, in color, in HTML. Using IPython utilities and pandas

In [ ]:
import pandas as pd
from textattack.loggers import CSVLogger
from textattack.attack_results import SuccessfulAttackResult

# pd.options.display.max_colwidth = (
#     480  # increase colum width so we can actually read the examples
# )

logger = CSVLogger(color_method="html")

for result in attack_results:
    if isinstance(result, SuccessfulAttackResult):
        logger.log_attack_result(result)

from IPython.core.display import display, HTML

results = pd.DataFrame.from_records(logger.row_list)
display(HTML(results[["original_text", "perturbed_text"]].to_html(escape=False)))

# Example 2: Attack from scratch


In [ ]:
# Load the dataset
from textattack.datasets import HuggingFaceDataset

dataset = HuggingFaceDataset("ag_news", None, "test")

In [ ]:
# Load the model
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained(  # loads a pre-trained BERT-based model for sequence classification from the HuggingFace model hub
    "textattack/bert-base-uncased-ag-news"
)
tokenizer = transformers.AutoTokenizer.from_pretrained( # loads the corresponding tokenizer for the BERT model
    "textattack/bert-base-uncased-ag-news"
)

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

### Banana word swap
We're going to try a very simple transformation: one that replaces any given word with the word 'banana'. In TextAttack, there's an abstract WordSwap class that handles the heavy lifting of breaking sentences into words and avoiding replacement of stopwords. We can extend WordSwap and implement a single method, _get_replacement_words, to indicate to replace each word with 'banana'. 🍌

In [ ]:
from textattack.transformations import WordSwap

class BananaWordSwap(WordSwap):
    """Transforms an input by replacing any word with 'banana'."""

    def _get_replacement_words(self, word):
        """Returns a list with one item, since `_get_replacement_words` is intended to return a list of candidate replacement words."""
        return ["banana"]

In [ ]:
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import (
    RepeatModification,   # constraint disallowing the modification of words which have already been modified
    StopwordModification, # constraint disallowing the modification of stopwords
)
from textattack import Attack

# Create the goal function using the model
goal_function = UntargetedClassification(model_wrapper)

# Use Banana word swap class as the attack transformation
transformation = BananaWordSwap()

# Avoid modification of already modified indices and stopwords
constraints = [RepeatModification(), StopwordModification()]

# Use the Greedy search method
search_method = GreedySearch()

# Create the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

In [ ]:
print(attack)

In [ ]:
from tqdm import tqdm
from textattack import Attacker
from textattack import AttackArgs

attack_args = AttackArgs(num_examples=10)

attacker = Attacker(attack, dataset, attack_args)

attack_results = attacker.attack_dataset()

In [ ]:
import pandas as pd
from textattack.loggers import CSVLogger
from textattack.attack_results import SuccessfulAttackResult

# pd.options.display.max_colwidth = (
#     480  # increase colum width so we can actually read the examples
# )

logger = CSVLogger(color_method="html")

for result in attack_results:
    if isinstance(result, SuccessfulAttackResult):
        logger.log_attack_result(result)

from IPython.core.display import display, HTML

results = pd.DataFrame.from_records(logger.row_list)
display(HTML(results[["original_text", "perturbed_text"]].to_html(escape=False)))

Looks like some examples needed only a couple "banana"s, while others needed up to 17 "banana" substitutions to change the class score.